In [1]:
import sys#TylerK: Add sys to get cmdline_helper from NRPy top directory; remove this line and next when debugged
sys.path.append('../')

import cmdline_helper as cmd     # NRPy+: Multi-platform Python command-line interface

# Create C code output directory:
Ccodesdir = "SEOBNR_Playground_Pycodes"

import numpy as np
import matplotlib.pyplot as plt
import scipy.integrate as spint
import Hamiltonian.Hreal_and_csi_on_bottom as hr
import Derivatives.ham_and_csi_first_derivs as dh
import Flux.flux as fl


# Tutorial - Right-Hand-Side's for Hamilton's Equation of Motion for SEOBNR Models

In this notebook, we will implement the right hand side functions for the Hamilton's EOMs so that we can integrate them to get the trajectory. Since we are using scipy's ODE integration functionality, we will be writing these as wrappers that will ultimately be passed into the scipy functions.

To begin, we will recap the ODEs that form the Hamilton's EOMs and document the individual terms. We then create the wrapper function that will pass the phase space variables and parameters to the derivative functions generated before.

## Step : Hamilton's EOMs for SEOBNR implementations

The Hamilton's EOMs for a general SEOBNR implementation is given in https://arxiv.org/pdf/0912.3466.pdf as a set of derivatives for the positions, momenta and spins.
We begin by stating the positional derivatives as

$$
\frac{dX^i}{dt} = T^i_j \frac{\partial \mathcal{H}_\rm{real}}{\partial P^*_j}
$$

where $P^*$ represent the tortoise momenta and the matrix $\mathbb{T}$ represents the transformation matrix from the tortoise momentum to the conjugate momentum. The momentum derivatives, on the other hand, are given as

$$
\frac{dP^*_i}{dt} = -T_i^j\frac{\partial \mathcal{H}_\rm{real}}{\partial X^j} + \frac{1}{\Omega L }\frac{dE}{dt}P^*_j + P^*_l\left(T^{-1}\right)_k^l\frac{\partial T_i^k}{\partial X^j}T_m^j\frac{\partial \mathcal{H}_\rm{real}}{\partial P^*_m} 
$$

where $\Omega$ is the rotational frequency, $L$ is the magnitude of the angular momentum and $\frac{dE}{dt}$ is the energy flux due ot gravitational waves. Since we already have dedicate functions to compute the Hamiltonian derivates and the fluxes, in this notebook we will document the terms unique to the RHS computations and call the dedicated functions as necessary.

Finally, the spin derivatives are given as (where $a$ labels the individual spin vectors so $a \in \{1,2\}$)

$$
\dot{\bf{S}}\bf{_a} = \{\bf{S_a},\mathcal{H}\} = \frac{\partial \mathcal{H}}{\partial \bf{S_a}}\times\bf{S_a}
$$


### The Angular Momentum Magnitude $L$

The magnitude of the angular momentum vector is defined as follows

$$
L = \sqrt{\bf{L}\cdot\bf{L}}
$$

### The Angular Momentum Vector $\bf{L}$

The angular momentum vector has its usual definition and is defined using the tortoise momentum due to (Eq.A1 of PRD 81, 084041 (2010))

$$
\bf{L} = \bf{X}\times\bf{P} = \bf{X}\times\bf{P^*}
$$

### The Rotational Frequence $\Omega$

The rotational frequency is defined here as

$$
\Omega = \frac{1}{R^2}\sqrt{\bf{X}\times\dot{\bf{X}}}
$$

Where $R$ is the radial distance and $\dot{\bf{X}}$ is computed as the first set of right hand sides.


### The Positional Derivatives $\dot{\bf{X}}$

The derivatives of the coordinates are given as:

$$
\frac{dX^i}{dt} = T^i_j \frac{\partial \mathcal{H}_\rm{real}}{\partial P^*_j}
$$

### The $\mathbb{T}$ matrix

The trnasformation matrix from tortoise to conjugate momentum are given as

$$
T_i^j = \delta_{ij} + \frac{X^iX^j}{R^2}\left[\xi_a - 1\right]
$$

while the inverse matrix is given as

$$
\left(T^{-1}\right)_i^j = \delta_{ij} - \frac{\xi_a - 1}{\xi_a}\frac{X^iX^j}{R^2} 
$$

and the derivative of the matrix is given as

$$
\frac{\partial T_i^j}{\partial X^k} = \frac{\delta_{ik}X^j + X^i\delta_{jk}}{R^2}\left(\xi_a - 1\right) - 2\frac{X^iX^jX^k}{R^3}\left(\xi_a - 1\right) + \frac{X^iX^j}{R^2}\frac{\partial\xi_a}{\partial X^k} 
$$

Where, $\xi_a$ is the tortoise factor defined below and $R$ is the radial distance

### The Radial Distance $R$

The radial distance $R$ is given as usual

$$
R = \sqrt{\bf{X}\cdot\bf{X}}
$$


In [2]:
def SEOBNR_RHS(t,var,m1,m2,EMGamma,tortoise):
    
    ## Unwrap all arguments
    eta = m1*m2/(m1+m2)/(m1+m2)
    q = np.array([var[0], var[1], var[2]])
    p = np.array([var[3], var[4], var[5]])
    S1 = np.array([var[6], var[7], var[8]])
    S2 = np.array([var[9], var[10], var[11]])
    
    ## Call necessary functions
    ham_and_potentials = hr.compute_Hreal_and_csi(m1,m2,EMGamma,tortoise,var[0],var[1],var[2],var[3],var[4],var[5],var[6],var[7],var[8],var[9],var[10],var[11])
    Hreal = ham_and_potentials[0]
    
    dHreal = dh.ham_first_derivs(m1,m2,tortoise,var[0],var[1],var[2],var[3],var[4],var[5],var[6],var[7],var[8],var[9],var[10],var[11])
    dHdX = np.array([dHreal[0],dHreal[1],dHreal[2]])/eta
    dHdP = np.array([dHreal[3],dHreal[4],dHreal[5]])/eta
    dHdS1 = np.array([dHreal[6],dHreal[7],dHreal[8]])
    dHdS2 = np.array([dHreal[9],dHreal[10],dHreal[11]])
    
    #for i in range(3):
    #    print("dHdP[%d] = %.16e"%(i,dHdP[i]))
    
    ## Compute R, R^-1 and R^-2
    
    R = np.linalg.norm(q)
    Rinv = 1/R
    R2inv = Rinv*Rinv
    
    ## Compute xi_a and xi_a - 1 and xi_a^-1
    
    xi_a = ham_and_potentials[1]
    xi_a_minusone = xi_a - 1
    xi_a_inv = 1/xi_a
    dxi_a = np.array([dHreal[12],dHreal[13],dHreal[14]])
    
    ## Assign and Populate the Tortoise Matrix, its inverse, and its derivative
    
    kron_delta = lambda x,y: 1 if x== y else 0 
    
    T = np.zeros([3,3])
    Tinv = np.zeros([3,3])
    dT = np.zeros([3,3,3])
    
    ## T matrix has been validated
    for i in range(3):
        for j in range(3):
            T[i,j] = kron_delta(i,j) + q[i]*q[j]*xi_a_minusone*R2inv
            Tinv[i,j] = kron_delta(i,j) - q[i]*q[j]*xi_a_minusone*R2inv*xi_a_inv
    
    #for i in range(3):
    #    for j in range(3):
    #        print("T[%d,%d] = %.16e"%(i,j,T[i,j]))
            
    #for i in range(3):
    #    for j in range(3):
    #        print("Tinv[%d,%d] = %.16e"%(i,j,Tinv[i,j]))
                
    for i in range(3):
        for j in range(3):
            for k in range(3):
                dT[i,j,k] = kron_delta(i,k)*q[j]*xi_a_minusone + q[i]*kron_delta(j,k) - 2*q[i]*q[j]*q[k]*Rinv*xi_a_minusone + q[i]*q[j]*dxi_a[k] 
        
    ## Compute the X derivatives
    
    dX = np.dot(T,dHdP)
    
    ## Compute Omega
    
    Omega = np.linalg.norm(np.cross(q,dX))*R2inv
    
    ## Compute Angular Momentum and its magnitude, L
    
    Lvec = np.cross(q,p)
    L = np.linalg.norm(Lvec)
    
    ## Call the flux function to get dEdt
    
    dEdt = fl.compute_flux(m1,m2,EMGamma,tortoise,q,p,S1,S2,Omega,Hreal)
    
    ## Compute the S1, S2 derivatives
    
    dS1 = np.cross(dHdS1,S1)
    dS2 = np.cross(dHdS2,S2)
    
    ## Compute the terms in the momentum derivative and compute the P derivative
    
    Ham_deriv_term_1 = - np.dot(T,dHdX)
    
    Ham_deriv_term_2 = np.zeros(3)
    ## no nested loop, define individual terms
    #for i in range(3):
    #    for j in range(3):
    #        for k in range(3):
    #            for l in range(3):
    #                for m in range(3):
    #                    Ham_deriv_term_2[i] += p[l]*Tinv[l,k]*dT[k,i,j]*T[j,m]*dHdP[m]
    
    Ham_deriv_term_2_T11 = np.zeros([3,3,3])
    Ham_deriv_term_2_T12 = np.zeros([3,3])
    Ham_deriv_term_2_T2 = np.zeros([3])
    
    for i in range(3):
        for j in range(3):
            for l in range(3):
                for k in range(3):
                    Ham_deriv_term_2_T11[i,j,l] += dT[i,k,j]*Tinv[k,l]
    
    for i in range(3):
        for j in range(3):
            for k in range(3):
                Ham_deriv_term_2_T12[i,j] += Ham_deriv_term_2_T11[i,j,k]*p[k]
    
    for i in range(3):
        for j in range(3):
            Ham_deriv_term_2_T2[i] += dHdP[j]*T[i,j]
    
    for i in range(3):
        for j in range(3):
            Ham_deriv_term_2[i] += Ham_deriv_term_2_T12[i,j]*Ham_deriv_term_2_T2[j]
                
    pstar = np.zeros(3)
    for i in range(3):
        for j in range(3):
            pstar[i] += T[i,j]*p[j]
    flux_term = pstar*dEdt/Omega/L/eta
    
    print(flux_term)
    print(Hreal)
    print(Omega)
    print(dEdt)
    
    dP = Ham_deriv_term_1 + Ham_deriv_term_2 + flux_term
    
    ## Return the values in the right order dX,dP,dS1,dS2
    
    return np.array([dX[0],dX[1],dX[2],dP[0],dP[1],dP[2],dS1[0],dS1[1],dS1[2],dS2[0],dS2[1],dS2[2]])


## Testing the RHS with the Initial Conditions

Here, we will plug into the RHS routine the initial conditions obtained from LALSuite in order to check if the function yields the correct right hand sides

In [3]:
lal_init_params = {'m1': 2.3000000000000000e+01, 'm2': 1.0000000000000000e+01, 'x': 2.1296906518814026e+01, 'y': 0.0000000000000000e+00, 'z': 0.0000000000000000e+00, 'S1x':  4.8576675849403119e-03, 'S1y': -9.7153351698806220e-03, 'S1z': -1.4573002754820936e-02, 'S2x': 3.6730945821854912e-03, 'S2y': -4.5913682277318639e-03, 'S2z': 5.5096418732782379e-03, 'p1': -3.3421692448608954e-04, 'p2': 2.3354019734342338e-01, 'p3': -1.1801419834967834e-04,'tortoise': 1}
Cartesianderivatives = [-3.1189346881256076e-04,2.1750028741859084e-01,-1.7796556642610693e-11,-2.3850891376906115e-03,-1.2922301422187956e-06,7.7822701783359718e-07,2.7612975913411751e-06,1.4003022311788185e-06,-1.3102290338820473e-08,2.6598842861765751e-06,2.1049446101261840e-06,-1.9135682345896791e-08,2.9576047184259879e-09,1.0212761381030454e-02]
MomentumTerm1 = [ -2.1654603268825789e-03 , -3.3210264168322584e-07 , 7.7756689975672089e-07 ]
MomentumTerm2 = [ -2.1963068026650492e-04 , 3.4619084687379877e-07 , -2.8326422409206953e-17 ]
FluxTerm = [ 1.8694584719772866e-09 , -1.3063183474093685e-06 , 6.6011810520264852e-10]
dEdt = 2.8412465229810723e-07
def SDA(x1,x2):
    erel = abs((x1 - x2)/(0.5*(x1 + x2)))
    return int(-np.log10(erel) + 1)



m1 = lal_init_params['m1']
m2 = lal_init_params['m2']
EMGamma = 0.577215664901532860606512090082402431
tortoise = lal_init_params['tortoise']
lal_init_conditions = np.array([lal_init_params['x'], lal_init_params['y'], lal_init_params['z'], lal_init_params['p1'], lal_init_params['p2'], lal_init_params['p3'], lal_init_params['S1x'], lal_init_params['S1y'], lal_init_params['S1z'], lal_init_params['S2x'], lal_init_params['S2y'], lal_init_params['S2z']])
rhs_first_step = SEOBNR_RHS(0,lal_init_conditions,m1,m2,EMGamma,tortoise)

for i in range(12):
    print("dvalues[%d] (lal, nrpy, sda) = %.16e , %.16e , %d"%(i,Cartesianderivatives[i],rhs_first_step[i],SDA(Cartesianderivatives[i],rhs_first_step[i])))


input params for compute_flux: 23.0 10.0 0.5772156649015329 1 [21.29690652  0.          0.        ] [-3.34216924e-04  2.33540197e-01 -1.18014198e-04] [ 0.00485767 -0.00971534 -0.014573  ] [ 0.00367309 -0.00459137  0.00550964] 0.01021276433878402 0.9952429832739629 1.0 8
l = 2, m = 1, hLM = -1.9013876315305334e-07
l = 2, m = 2, hLM = 1.3059115259222284e-04
l = 3, m = 1, hLM = 1.1061284502858760e-08
l = 3, m = 2, hLM = 1.4428353034881475e-09
l = 3, m = 3, hLM = -3.9104860436366075e-07
l = 4, m = 1, hLM = 1.1470444824949251e-13
l = 4, m = 2, hLM = -1.6194518739552105e-10
l = 4, m = 3, hLM = -8.6045339914848297e-12
l = 4, m = 4, hLM = 3.4965893871558898e-09
l = 5, m = 1, hLM = -4.7677787124586631e-15
l = 5, m = 2, hLM = -2.1469342984924391e-15
l = 5, m = 3, hLM = 1.3030599802936452e-12
l = 5, m = 4, hLM = 6.2078579046551827e-14
l = 5, m = 5, hLM = -2.2848694497949642e-11
l = 6, m = 1, hLM = -3.2641786519740556e-20
l = 6, m = 2, hLM = 1.6411775303558390e-16
l = 6, m = 3, hLM = 2.38641812655

In [4]:
## print out tortoise to make sure all inputs are consistent
## toggle the inputs to see which terms are problematic

In [6]:
## simply ported final initial conditions from our lal solver

#var_init = np.array([ 2.1296906518814026e+01,0.000000000000,0.000000000000,-3.3421692448608954e-04,2.3354019734342338e-01,-1.1801419834967834e-04,4.8576675849403119e-03,-9.7153351698806220e-03,-1.4573002754820936e-02,3.6730945821854912e-03,-4.5913682277318639e-03,5.5096418732782379e-03])
#params = (23,10,0.577215664901532860606512090082402431,1)
#max_time = 3.

#result = spint.solve_ivp(SEOBNR_RHS,(0,max_time),var_init,method = 'RK45',dense_output=True,args = params)

#dt = 1
#time_series = np.arange(0.,max_time,dt)
#print(time_series)

#sparse_array = result.y
#dense_sol = result.sol.__call__(time_series)

#plot_array = np.array([dense_sol[:,i] for i in range(len(time_series))])

#plt.plot(plot_array[:,0],plot_array[:,1])
#plt.axis('square')
#print(sparse_array)


In [ ]:
#plt.plot(plot_array[:,0],plot_array[:,1])
#plt.axis('square')

In [ ]:
#data = "outputv4Pindex0.txt"
#datapert = "outputv4Pindex0pert.txt"
#LALvalues = []
#LALvaluespert = []
#eta = (m1*m2)/(m1+m2)/(m1+m2)

#with open(data) as file, open(datapert) as filepert:
#    data_lines = file.readlines()
#    pertdata_lines = filepert.readlines()
#    for i in range(len(data_lines)//2):
#        valuepairs = data_lines[2*i].strip('{}\n').split(',')
#        pertvaluepairs = pertdata_lines[2*i].strip('{}\n').split(',')
#        values = dict()
#        pertvalues = dict()
#        #print(i)
#        for j in range(len(valuepairs)):
#            pair = valuepairs[j].split(': ')
#            pertpair = pertvaluepairs[j].split(': ')
            #print(pertpair)
#            values[pair[0].strip(" ''")] = float(pair[1])
#            pertvalues[pertpair[0].strip(" ''")] = float(pertpair[1])
#        LALvalues.append([values['x'],values['y']])
#        LALvaluespert.append([pertvalues['x'],pertvalues['y']])


#plt.plot(plot_array[:,0],plot_array[:,1])
#LALvalues = np.array(LALvalues)
#LALvaluespert = np.array(LALvaluespert)
#plt.plot(LALvalues[:,0],LALvalues[:,1])
#plt.plot(LALvaluespert[:,0],LALvaluespert[:,1])
#plt.axis('square')




In [ ]:
#plt.plot(plot_array[:,0],plot_array[:,1])
#dt = 1
#time_series = np.arange(0.,max_time,dt)
#print(time_series)

#sparse_array = result.y
#dense_sol = result.sol.__call__(time_series)

#plot_array = np.array([dense_sol[:,i] for i in range(len(time_series))])

#LALvalues = np.array(LALvalues)
#plt.plot(np.log10(np.abs((LALvalues[:,0] - plot_array[:,0])/LALvalues[:,0])))
#plt.plot(np.log10(np.abs((LALvaluespert[:,0]- LALvalues[:,0])/LALvalues[:,0])))
#plt.plot(LALvalues[:,0])
#plt.plot(plot_array[:,0])

In [ ]:
#plt.plot(LALvalues[:,1])
#plt.plot(plot_array[:,1])

In [ ]:
#plt.plot([np.linalg.norm([plot_array[i,0],plot_array[i,1]]) for i in range(len(plot_array))])
#plt.plot([np.linalg.norm([LALvalues[i,0],LALvalues[i,1]]) for i in range(len(plot_array))])
